# Segmenting and Clustering Neighbourhoods in Toronto

## Question 1 - Data Collection
1. Start by creating a new Notebook for this assignment.
2. Hints for scraping Notebook
3. To create the dataframe
4. Submit a link to your Notebook on your Github repository.(10 marks)

In [1]:
# Import packages
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Import geocoder for Question 2
import geocoder # import geocoder

In [5]:
# Scraping from Wiki
url_canada = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
src_canada = requests.get(url_canada).text
can_soup = BeautifulSoup(src_canada,'lxml')
# can_soup.table

In [24]:
# Orginizing and cleaning data
table_contents = []
can_table = can_soup.find('table')
for row in can_table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
list_postal = pd.DataFrame(table_contents)
list_postal['Borough'] = list_postal['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                                         'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                                         'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                                         'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

# print(list_postal['Borough'].unique())
# print(list_postal['Neighborhood'].unique())
# list_postal
list_postal.shape

(103, 3)

In [25]:
# Testing
# Containing "Not assigned" Borough?
df_clean1 = list_postal[list_postal['Borough'] != 'Not assigned']
print(df_clean1.shape)

# Containing duplicated PostalCode/Borough
df_clean2 = df_clean1.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df_clean2.reset_index(inplace=True)
print(df_clean2.shape)

(103, 3)
(103, 3)


In [26]:
# list_postal
list_postal

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Question 2
Merge Coords to Postal Codes

In [52]:
# we need to get the latitude and the longitude coordinates of each neighborhood. 
# Obtain unique postal codes
postals = list_postal['PostalCode'].unique()
postals 

# Obtain postal code <-> lat/lng crosswalk
geo_postals = []
for postal_code in postals:
    geo = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng = geo.latlng
    # lat = lat_lng[0]
    # lng = lat_lng[1]
    
    row = [postal_code, lat_lng]
    geo_postals.append(row)

geo_postals
# Not able to use GOOGLE anymore, OSM obtain wrong LAT/LNG

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [57]:
# USE PROVIED CROSSWALK INSTEAD
geo_coords = pd.read_csv('./Data/Geospatial_Coordinates.csv')
geo_coords

geo_coords.rename(columns={'Postal Code':'PostalCode'},inplace=True)
can_merged = pd.merge(list_postal, geo_coords, on='PostalCode')
print(can_merged.shape)

can_merged

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
